In [30]:
import pandas as pd
import numpy as np

np.random.seed(0)

def generate_item_df(num_items = 20, weight_range = (0, 10), value_range = (0, 1)):
    weights = np.random.uniform(*weight_range, num_items)
    values = np.random.uniform(*value_range, num_items)
    return pd.DataFrame({
        'Item': [f'item {i}' for i in range(num_items)],
        'Weight': weights,
        'Value': values
    })

df = generate_item_df()
print(df.head(4))

     Item    Weight     Value
0  item 0  5.488135  0.978618
1  item 1  7.151894  0.799159
2  item 2  6.027634  0.461479
3  item 3  5.448832  0.780529


In [31]:
import random

random.seed(0)

def genetic_algorithm(df, max_weight=30, population_size=100, num_generations=50, mutation_rate=0.01):
    weights = df["Weight"].tolist()
    values = df["Value"].tolist()
    n = len(values)

    average_weight = sum(weights) / n
    expected_items = max_weight / average_weight
    bit_flip_probability = expected_items / n

    # Initialise the population so that each individual has roughly the correct weight
    population = [[1 if random.random() < bit_flip_probability else 0 for _ in range(n)]
                  for _ in range(population_size)]

    def fitness(individual):
        total_value = sum(value if individual[i] else 0 for i, value in enumerate(values))
        total_weight = sum(weight if individual[i] else 0 for i, weight in enumerate(weights))
        if (diff := max_weight - total_weight) < 0:
            # The more overweight we are, the more negative the fitness is
            return diff
        return total_value

    def tournament_selection(population, k=3):
        return max(random.sample(population, k), key=fitness)

    def crossover(parent1, parent2):
        crossover_point = random.randint(1, len(parent1) - 1)
        return parent1[:crossover_point] + parent2[crossover_point:], parent2[:crossover_point] + parent1[crossover_point:]

    def mutate(individual):
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = 1 - individual[i] # Flip bit
        return individual

    for _ in range(num_generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(mutate(offspring1))
            new_population.append(mutate(offspring2))
        population = new_population

    best = max(population, key=fitness)
    return best

In [32]:
def test_genetic_algorithm(df, n_runs=4):
    for _ in range(n_runs):
        indices = [i for i, bit in enumerate(genetic_algorithm(df)) if bit]
        items = [df.iloc[i] for i in indices]
        print(f"Items: {', '.join(map(str, indices))}")
        print(f"Total weight: {sum([item['Weight'] for item in items]):.2f}")
        print(f"Total value: {sum([item['Value'] for item in items]):.2f}")
        print()


In [33]:
# Test with the original 20 items
test_genetic_algorithm(df)

Items: 0, 3, 9, 11, 14, 15, 16, 18
Total weight: 29.63
Total value: 5.14

Items: 0, 3, 7, 14, 15, 16, 18
Total weight: 29.42
Total value: 4.90

Items: 0, 3, 14, 15, 16, 18, 19
Total weight: 29.20
Total value: 4.63

Items: 0, 3, 9, 11, 14, 15, 16, 18
Total weight: 29.63
Total value: 5.14



In [34]:
# Test with 1000 items
test_genetic_algorithm(generate_item_df(1000))
# Very overweight - TODO: fix this
# Also very slow

Items: 1, 3, 6, 7, 13, 29, 42, 68, 70, 75, 78, 80, 89, 92, 93, 100, 101, 113, 118, 126, 128, 136, 138, 139, 149, 150, 151, 155, 164, 174, 177, 180, 182, 185, 194, 198, 204, 209, 213, 219, 222, 245, 250, 255, 260, 263, 269, 275, 278, 299, 301, 311, 312, 317, 322, 341, 348, 349, 354, 358, 359, 362, 370, 377, 382, 383, 389, 406, 423, 427, 429, 440, 453, 456, 458, 466, 468, 477, 479, 480, 509, 513, 514, 516, 520, 521, 523, 530, 532, 533, 536, 545, 546, 558, 560, 563, 564, 567, 574, 575, 577, 578, 580, 588, 591, 598, 607, 609, 612, 613, 615, 617, 620, 624, 631, 638, 648, 649, 651, 657, 660, 665, 666, 668, 670, 671, 682, 684, 698, 702, 704, 705, 710, 717, 723, 733, 738, 740, 743, 745, 751, 753, 757, 767, 775, 780, 783, 789, 794, 802, 803, 807, 819, 827, 828, 831, 834, 835, 857, 874, 880, 887, 900, 901, 908, 912, 921, 928, 936, 943, 944, 945, 950, 958, 959, 961, 962, 968, 976, 981, 987
Total weight: 634.99
Total value: 94.77

Items: 3, 12, 20, 36, 37, 40, 45, 48, 59, 64, 70, 83, 84, 89, 92, 1